# Province and Territory-wise Scraping

In [ ]:
import sys
import requests
import urllib.request
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import csv
import time
import pprint

In [ ]:
my_url = 'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&sort=provincial'
my_url

In [ ]:
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

page_soup = soup(page_html, "html.parser")

unorderedList = page_soup.findAll("ul",{"class":"clearfix list-group"})

RowList = []
RowList.append(['Province Names', 'Links'])

for i in range(len(unorderedList)):
    ul = unorderedList[i].findAll("a")
    for j in range(len(ul)):
        RowList.append([ul[j].text, "http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&sort=provincial"+ul[j]['href']])
RowList

In [ ]:
with open('Data 501 outputs/Province and Territory/SayersProvinceName.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(RowList)
csvFile.close()

# Second Phase of Scraping

### The following 3 blocks do not work;

In [ ]:
# Importing the dependencies
# This is needed to create a lxml object that uses the css selector
from lxml.etree import fromstring
  
# The requests library
import requests

import cssselect

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

class ElectionDataScraper:
  
    API_url = 'http://canadianelectionsdatabase.ca/PHASE5/request.controller.php'
    scraped_elections = []

    def get_elections_info(self, pid):

        # This is the only data required by the api 
        # To send back the stores info
        data = {
        # '#page_1': 'province_id-equals-'+str(pid)
            "search":{"type":"page_1","value":"province_id-equals-9"},
            "search_type":"province",
            "previous_title":"Alberta"
        }
        # Making the post request
        response = requests.post(self.API_url, data=data)

        # The data that we are looking is in the second
        # Element of the response and has the key 'data', 
        # so that is what's returned
        return response.json()
    
    def parse_elections_info(self, data):
    # Creating an lxml Element instance
        element = fromstring(data)

        for store in element.cssselect('.sorted'):
            # We add the store to the scraped stores list
            self.scraped_elections.append(store)
    
    def run(self):
        for pid in range(14):
            # Retrieving the data
            data = self.get_elections_info(pid)
            pp.pprint(data)
            # self.parse_elections_info(data)
            print('scraped the page' + str(pid))
        # self.save_data()
    
    def save_data(self):
        with open('elections_info.json', 'w') as json_file:
            json.dump(self.scraped_elections, json_file, indent=4)

In [ ]:
if __name__ == '__main__':
  scraper = ElectionDataScraper()
  scraper.run()

## Selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0

In [ ]:
times = [20, 50]
time = times[0]
i = 0

RowList = [['Election Date(Province Initial)','Link to the page']]
while i < 14:
    if i == 3:
        i = i+1
        continue
    try:
        with webdriver.Chrome('C:/Users/moham/Downloads/chromedriver.exe') as driver:
            wait = WebDriverWait(driver, time)
            driver.get('http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&sort=provincial#page_1=province_id-equals-'+str(i))
            first_result = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sorted")))
            for j in range(1,7):
                link = first_result.find_element_by_xpath('//*[@id="tree-stage"]/div/div['+str(j)+']/a').get_attribute('href')
                inner_html = first_result.find_element_by_xpath('//*[@id="tree-stage"]/div/div['+str(j)+']/a').get_attribute('innerHTML')
                print(i, j, inner_html, link)
                RowList.append([inner_html, link])
        i = i+1
    except:
        print("error occured")
        if time == times[0]:
            time = times[1]
        else:
            time = times[0]
RowList

In [ ]:
# with open('Data 501 outputs/Province and Territory/SayersProvinceName_ElectionDataLinks.csv', 'w', newline='') as csvFile:
with open('SayersProvinceName_ElectionDataLinks.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(RowList)
csvFile.close()

### Result RowList 
= [['Election Date(Province Initial)', 'Link to the page'],
 ['June 7 2018 (ON)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=786'],
 ['June 12 2014 (ON)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=686'],
 ['October 6 2011 (ON)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=685'],
 ['October 10 2007 (ON)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=684'],
 ['October 2 2003 (ON)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=683'],
 ['June 3 1999 (ON)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=682'],
 ['April 23 2019 (PE)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=788'],
 ['May 4 2015 (PE)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=763'],
 ['October 3 2011 (PE)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=430'],
 ['May 28 2007 (PE)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=431'],
 ['September 29 2003 (PE)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=432'],
 ['April 17 2000 (PE)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=429'],
 ['May 30 2017 (NS)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=785'],
 ['October 8 2013 (NS)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=756'],
 ['June 9 2009 (NS)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=476'],
 ['June 13 2006 (NS)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=475'],
 ['August 5 2003 (NS)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=474'],
 ['July 27 1999 (NS)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=473'],
 ['September 24 2018 (NB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=792'],
 ['September 22 2014 (NB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=755'],
 ['September 27 2010 (NB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=528'],
 ['September 18 2006 (NB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=278'],
 ['June 9 2003 (NB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=527'],
 ['June 7 1999 (NB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=526'],
 ['May 16 2019 (NL)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=793'],
 ['November 30 2015 (NL)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=780'],
 ['October 11 2011 (NL)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=758'],
 ['October 9 2007 (NL)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=410'],
 ['October 21 2003 (NL)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=409'],
 ['February 9 1999 (NL)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=411'],
 ['October 3 2011 (NWT)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=708'],
 ['October 1 2007 (NWT)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=707'],
 ['November 24 2003 (NWT)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=706'],
 ['December 6 1999 (NWT)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=705'],
 ['October 16 1995 (NWT)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=704'],
 ['October 15 1991 (NWT)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=703'],
 ['April 19 2016 (MB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=764'],
 ['October 4 2011 (MB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=385'],
 ['May 22 2007 (MB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=364'],
 ['June 3 2003 (MB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=361'],
 ['September 21 1999 (MB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=360'],
 ['April 25 1995 (MB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=358'],
 ['April 4 2016 (SK)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=779'],
 ['November 7 2011 (SK)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=566'],
 ['November 7 2007 (SK)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=567'],
 ['November 5 2003 (SK)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=568'],
 ['September 16 1999 (SK)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=569'],
 ['June 21 1995 (SK)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=570'],
 ['April 16 2019 (AB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=787'],
 ['May 5 2015 (AB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=753'],
 ['April 23 2012 (AB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=749'],
 ['March 3 2008 (AB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=274'],
 ['November 22 2004 (AB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=276'],
 ['March 12 2001 (AB)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=275'],
 ['May 9 2017 (BC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=784'],
 ['May 14 2013 (BC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=750'],
 ['May 12 2009 (BC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=412'],
 ['May 17 2005 (BC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=383'],
 ['May 16 2001 (BC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=382'],
 ['May 28 1996 (BC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=381'],
 ['October 1 2018 (QC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=794'],
 ['April 7 2014 (QC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=761'],
 ['September 4 2012 (QC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=760'],
 ['December 8 2008 (QC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=565'],
 ['March 26 2007 (QC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=564'],
 ['April 14 2003 (QC)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=563'],
 ['October 30 2017 (NU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=782'],
 ['February 10 2014 (NU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=783'],
 ['October 28 2013 (NU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=747'],
 ['October 27 2008 (NU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=746'],
 ['February 16 2004 (NU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=745'],
 ['February 15 1999 (NU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=326'],
 ['October 11 2011 (YU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=744'],
 ['October 10 2006 (YU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=743'],
 ['November 4 2002 (YU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=742'],
 ['April 17 2000 (YU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=741'],
 ['September 30 1996 (YU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=740'],
 ['October 19 1992 (YU)',
  'http://canadianelectionsdatabase.ca/PHASE5/?p=0&type=election&ID=739']]

# Third Phase of Scraping

In [ ]:
for j in range(1, len(RowList)):
    my_url = RowList[j][1]

    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()

    page_soup = soup(page_html, "html.parser")

    try:
        tbody = page_soup.findAll("table",{"class":"grid"})[1].findAll("tr")[1].findAll("td")
    except:
        print("error occured at row:",j)
        continue

    tempList = []
    for i in range(len(tbody)):
        tempList.append(tbody[i].text)

    RowList[j].extend(tempList)
RowList

In [ ]:
RowList[0] = ['Election Date(Province Initial)', 'Link to the page','Party','Candidates','Seats Won','% of Seats Won','% of Vote']
RowList

In [ ]:
# with open('Data 501 outputs/Province and Territory/SayersProvinceName_ElectionDataLinks_Party_Details.csv', 'w', newline='') as csvFile:
with open('SayersProvinceName_ElectionDataLinks_Party_Details.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(RowList)
csvFile.close()